## Import libraries

In [1]:
import os
import numpy as np
import tensorflow as tf 
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from qkeras import *
from qkeras import QActivation
from qkeras import QDense, QConv2DBatchnorm
import hls4ml

2025-12-08 19:58:01.869783: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-08 19:58:02.237903: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-08 19:58:02.239245: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-08 19:58:03.636917: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Path to Vitis HLS


Vitis HLS directory should be specified to use hls4ml with the high-level synthesis tool.

In [13]:
# PATH Xilinx Virtual Machine

os.environ['PATH'] = '/mnt/d/Xilinx/Vitis_HLS/2022.2/bin:' + os.environ['PATH']
os.environ['PATH'] = '/mnt/d/Xilinx/Vitis/2022.2/bin:' + os.environ['PATH']
os.environ['PATH'] = '/mnt/d/Xilinx/Vivado/2022.2/bin:' + os.environ['PATH']

## Load model

In [14]:
# Load keras model 
from qkeras.utils import _add_supported_quantized_objects
co = {}
_add_supported_quantized_objects(co)
model = load_model('../models/studentModelMnistCNN.h5', custom_objects=co)
model.summary()


c:\Users\fisica\Desktop\valentina\Guatemala\labs\lab5\Lab5_HLS4ML\neuralEnv\Lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer LecunUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Model: "studentQ_CNN_MLP"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (QConv2D)             (None, 26, 26, 16)        160       
                                                                 
 conv1_act (QActivation)     (None, 26, 26, 16)        0         
                                                                 
 pool1 (MaxPooling2D)        (None, 13, 13, 16)        0         
                                                                 
 conv2 (QConv2D)             (None, 11, 11, 32)        4640      
                                                                 
 conv2_act (QActivation)     (None, 11, 11, 32)        0         
                                                                 
 pool2 (MaxPooling2D)        (None, 5, 5, 32)          0         
                                                                 
 flatten (Flatten)           (None, 800)          

## hls4ml integration

**hls4ml** is a Python package developed for converting machine learning (ML) models into HLS (High-Level Synthesis) projects, enabling deployment of ML-based inference on hardware like FPGAs. More details can be found at [hls4ml documentation](https://fastmachinelearning.org/hls4ml/).

The user can control several options related to the model, including:

- **Precision:** Define the precision of the calculations in your model (e.g., fixed-point or floating-point representation).

- **Dataflow/Resource Reuse:** Control the level of parallelism or streaming in model implementations, with varying degrees of pipelining.

- **Quantization Aware Training:** Achieve optimized performance at low precision by using tools like QKeras. Models trained with QKeras benefit automatically from hls4ml's parsing of QKeras models during inference.

An HLS configuration should be created using the function `hls4ml.utils.config_from_keras_model(kerasModel, granularity)`, where kerasModel is the pre-trained model you want to implement on an FPGA, and granularity determines the configuration level. The two possible values for granularity are:

- 'model': The same configuration applies to the entire model (e.g., all layers use 16-bit fixed-point precision).

- 'name': Layer-specific configurations can be applied (e.g., the input layer can be defined in 8-bit fixed-point precision, while the second layer is set to 16-bit fixed-point precision).

In [15]:

hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')


from src import plotting

print("-----------------------------------")
plotting.print_dict(hls_config)
print("-----------------------------------")




Interpreting Sequential
Topology:
Layer name: inputLayer, layer type: InputLayer, input shapes: [[None, 28, 28, 1]], output shape: [None, 28, 28, 1]
Layer name: conv1, layer type: QConv2D, input shapes: [[None, 28, 28, 1]], output shape: [None, 26, 26, 16]
Layer name: conv1_act, layer type: Activation, input shapes: [[None, 26, 26, 16]], output shape: [None, 26, 26, 16]
Layer name: pool1, layer type: MaxPooling2D, input shapes: [[None, 26, 26, 16]], output shape: [None, 13, 13, 16]
Layer name: conv2, layer type: QConv2D, input shapes: [[None, 13, 13, 16]], output shape: [None, 11, 11, 32]
Layer name: conv2_act, layer type: Activation, input shapes: [[None, 11, 11, 32]], output shape: [None, 11, 11, 32]
Layer name: pool2, layer type: MaxPooling2D, input shapes: [[None, 11, 11, 32]], output shape: [None, 5, 5, 32]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 5, 5, 32]], output shape: [None, 800]
Layer name: fc1, layer type: QDense, input shapes: [[None, 800]], output s

In [16]:
hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')


Interpreting Sequential
Topology:
Layer name: inputLayer, layer type: InputLayer, input shapes: [[None, 28, 28, 1]], output shape: [None, 28, 28, 1]
Layer name: conv1, layer type: QConv2D, input shapes: [[None, 28, 28, 1]], output shape: [None, 26, 26, 16]
Layer name: conv1_act, layer type: Activation, input shapes: [[None, 26, 26, 16]], output shape: [None, 26, 26, 16]
Layer name: pool1, layer type: MaxPooling2D, input shapes: [[None, 26, 26, 16]], output shape: [None, 13, 13, 16]
Layer name: conv2, layer type: QConv2D, input shapes: [[None, 13, 13, 16]], output shape: [None, 11, 11, 32]
Layer name: conv2_act, layer type: Activation, input shapes: [[None, 11, 11, 32]], output shape: [None, 11, 11, 32]
Layer name: pool2, layer type: MaxPooling2D, input shapes: [[None, 11, 11, 32]], output shape: [None, 5, 5, 32]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 5, 5, 32]], output shape: [None, 800]
Layer name: fc1, layer type: QDense, input shapes: [[None, 800]], output s

In [17]:
for Layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][Layer]['Strategy'] = 'Latency'
    hls_config['LayerName'][Layer]['ReuseFactor'] = 1
    hls_config['LayerName'][Layer]['Precision'] = 'ap_fixed<8,4>'

hls_config['LayerName']['outputActivation']['Strategy'] = 'Stable'
hls_config['Model']['Precision'] = 'ap_fixed<16,6>'

hls_config['LayerName']['inputLayer']['Precision'] = 'ap_fixed<16, 6>'   


### hls4ml with Vitis HLS as backend

In [18]:
# Create configuration for Vitis HLS as backend.
cfg = hls4ml.converters.create_config(backend='Vitis')

# HLSConfig correspond to the configuration created in hls_config 
cfg['HLSConfig']  = hls_config
# Model to be converted
cfg['KerasModel'] = model
# Folder where the HLS project will be created
cfg['OutputDir']  = 'vitis_mnistCNN/'
# FPGA part 
cfg['Part'] = 'xc7z020clg400-1'  # PYNQ-Z1 or Zedboard: xc7z020clg484-1  
  
hls_model = hls4ml.converters.keras_to_hls(cfg)

hls_model.compile()

Interpreting Sequential
Topology:
Layer name: inputLayer, layer type: InputLayer, input shapes: [[None, 28, 28, 1]], output shape: [None, 28, 28, 1]
Layer name: conv1, layer type: QConv2D, input shapes: [[None, 28, 28, 1]], output shape: [None, 26, 26, 16]
Layer name: conv1_act, layer type: Activation, input shapes: [[None, 26, 26, 16]], output shape: [None, 26, 26, 16]
Layer name: pool1, layer type: MaxPooling2D, input shapes: [[None, 26, 26, 16]], output shape: [None, 13, 13, 16]
Layer name: conv2, layer type: QConv2D, input shapes: [[None, 13, 13, 16]], output shape: [None, 11, 11, 32]
Layer name: conv2_act, layer type: Activation, input shapes: [[None, 11, 11, 32]], output shape: [None, 11, 11, 32]
Layer name: pool2, layer type: MaxPooling2D, input shapes: [[None, 11, 11, 32]], output shape: [None, 5, 5, 32]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 5, 5, 32]], output shape: [None, 800]
Layer name: fc1, layer type: QDense, input shapes: [[None, 800]], output s

Done
"." no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.



Exception: Failed to compile project "myproject"

In [ ]:
### Usar en terminal el sigueinte comando
# $env:PATH += ";D:\Xilinx\Vitis_HLS\2022.2\bin"
# vitis_hls -version
# cd .\pythonModel\hlsPrj\
# vitis_hls -f .\build_prj.tcl

In [21]:
# This will perform the synthesis of the HLS project, showing the main results (latency and resource usage) once the process is completed. 

hls_model.build(csim=False, export=False)

CSynthesis report not found.
Vivado synthesis report not found.
Cosim report not found.
Timing report not found.


{}

----

Once this stage is complete, you can return to the wiki to continue with the next steps. 